In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

In [3]:
data = pd.read_excel('Data_v.5.xlsx')
data = data.copy()

In [4]:
data

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Lat,Long
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,-6.175392,106.827153
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,-6.137645,106.817125
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Hiburan,Jakarta,270000,4.6,-6.125312,106.833538
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Hiburan,Jakarta,10000,4.5,-6.302446,106.895156
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Hiburan,Jakarta,94000,4.5,-6.124190,106.839134
...,...,...,...,...,...,...,...,...,...
446,447,Teluk Kiluan,Teluk terkenal dengan atraksi lumba-lumba di L...,Alam,Tangamus,0,4.6,-5.779489,105.093217
447,448,Pantai Tanjung Setia,Pantai dengan ombak besar untuk berselancar di...,Alam,Pesisir Selatan,0,4.5,-5.302336,103.999850
448,449,Pantai Dewi Mandapa,Pantai dengan pemandangan indah dan fasilitas ...,Alam,Pesarawan,0,3.9,-5.572197,105.243705
449,450,Bendungan Margotirto,Bendungan dengan pemandanganAlam yang indah di...,Alam,Tangamus,0,4.3,-5.421126,104.725779


In [7]:
# Step 1: Data Preprocessing
le_category = LabelEncoder()
le_city = LabelEncoder()

# Fit the encoders with all unique categories and cities
le_category.fit(data['Category'].unique())
le_city.fit(data['City'].unique())

# Transform the actual data
data['Category'] = data['Category'].apply(lambda x: le_category.transform([x])[0] if x in le_category.classes_ else -1)
data['City'] = data['City'].apply(lambda x: le_city.transform([x])[0] if x in le_city.classes_ else -1)


# Handle unknown categories and cities
if -1 in data['Category'].values:
    max_category = data['Category'].max()
    data.loc[data['Category'] == -1, 'Category'] = max_category + 1
    le_category.classes_ = np.append(le_category.classes_, 'Unknown')

if -1 in data['City'].values:
    max_city = data['City'].max()
    data.loc[data['City'] == -1, 'City'] = max_city + 1
    le_city.classes_ = np.append(le_city.classes_, 'Unknown')

X = data[['Category', 'City', 'Rating', 'Price','Lat','Long']].values
#X = data[['Category', 'City', 'Rating']].values
scaler = MinMaxScaler()
X[:, 2:] = scaler.fit_transform(X[:, 2:])

In [9]:
# Step 2: Feature Encoding
num_categories = len(le_category.classes_)
num_cities = len(le_city.classes_)
X_encoded = np.zeros((X.shape[0], num_categories + num_cities + 4))
X_encoded[:, :num_categories] = tf.keras.utils.to_categorical(X[:, 0], num_classes=num_categories)
X_encoded[:, num_categories:num_categories+num_cities] = tf.keras.utils.to_categorical(X[:, 1], num_classes=num_cities)
X_encoded[:, -4:] = X[:, 2:]  # Rating and Price

# Step 3-4: Model Architecture and Training (same as before)
input_dim = X_encoded.shape[1]
hidden_dim = 64
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_dim, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dense(hidden_dim, activation='relu'),
    tf.keras.layers.Dense(input_dim, activation='sigmoid')
])
model.compile(optimizer='adam', loss='mse')

from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(X_encoded, test_size=0.2, random_state=42)
model.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_val, X_val), verbose=1)

Epoch 1/50
12/12 [==============================] - 2s 67ms/step - loss: 0.2365 - val_loss: 0.2217
Epoch 2/50
12/12 [==============================] - 0s 21ms/step - loss: 0.2064 - val_loss: 0.1800
Epoch 3/50
12/12 [==============================] - 0s 16ms/step - loss: 0.1517 - val_loss: 0.1096
Epoch 4/50
12/12 [==============================] - 0s 23ms/step - loss: 0.0789 - val_loss: 0.0487
Epoch 5/50
12/12 [==============================] - 0s 18ms/step - loss: 0.0368 - val_loss: 0.0306
Epoch 6/50
12/12 [==============================] - 0s 17ms/step - loss: 0.0272 - val_loss: 0.0272
Epoch 7/50
12/12 [==============================] - 0s 18ms/step - loss: 0.0250 - val_loss: 0.0259
Epoch 8/50
12/12 [==============================] - 0s 20ms/step - loss: 0.0238 - val_loss: 0.0249
Epoch 9/50
12/12 [==============================] - 0s 24ms/step - loss: 0.0228 - val_loss: 0.0239
Epoch 10/50
12/12 [==============================] - 0s 26ms/step - loss: 0.0217 - val_loss: 0.0231
Epoch 11/

In [10]:
# Step 5: Inference
def get_feature_vector(attraction_name):
    attraction = data[data['Place_Name'] == attraction_name].iloc[0]
    features = [
        attraction['Category'],
        attraction['City'],
        attraction['Rating'],
        attraction['Price'],
        attraction['Lat'],
        attraction['Long']
    ]
    
    encoded = np.zeros((1, input_dim))
    encoded[0, :num_categories] = tf.keras.utils.to_categorical(features[0], num_classes=num_categories)
    encoded[0, num_categories:num_categories+num_cities] = tf.keras.utils.to_categorical(features[1], num_classes=num_cities)
    encoded[0, -4:] = scaler.transform([features[2:]])
    
    return encoded


In [11]:
def get_recommendations(attraction_name, top_k=10):
    query_vector = get_feature_vector(attraction_name)
    query_embedding = model.predict(query_vector)
    all_embeddings = model.predict(X_encoded)
    similarities = np.dot(all_embeddings, query_embedding.T).flatten()
    top_indices = similarities.argsort()[::-1][0:top_k]  #Exclude the query itself
    query_index = np.where(top_indices == 0)[0]  # Find the index of the query
    top_attractions = data.iloc[top_indices]['Place_Name'].tolist()
    top_similarities = similarities[top_indices].tolist()
    
    recommendations = []
    for idx, (attraction, similarity) in enumerate(zip(top_attractions, top_similarities)):
        attraction_data = data[data['Place_Name'] == attraction].iloc[0]
        
        try:
            category = le_category.inverse_transform([attraction_data['Category']])[0]
        except ValueError:
            category = 'Unknown'
        
        try:
            city = le_city.inverse_transform([attraction_data['City']])[0]
        except ValueError:
            city = 'Unknown'
        
        recommendations.append({
            'rank': idx + 1,
            'name': attraction,
            'similarity': similarity,
            'category': category,
            'city': city,
            'rating': attraction_data['Rating'],
            'price': attraction_data['Price'],
            'lat': attraction_data['Lat'],
            'long': attraction_data['Long']
        })
    
    return recommendations

In [18]:
query_attraction = "Pantai Cibobos"  # Replace with an actual attraction from your dataset
recommendations = get_recommendations(query_attraction, top_k=10)
filtered_recommendations = [rec for rec in recommendations if rec["name"] != query_attraction]

print(f"\nTop 10 recommendations similar to {query_attraction} (excluding the same name):")
for rec in filtered_recommendations:
    print(f"{rec['rank']}. {rec['name']} (Similarity: {rec['similarity']:.3f})")
    print(f"   Category: {rec['category']}")
    print(f"   City: {rec['city']}")
    print(f"   Rating: {rec['rating']}")
    print(f"   Price: {rec['price']}")
    print(f"   Latitude: {rec['lat']}")
    print(f"   Longitude: {rec['long']}")
    print()

15/15 [==============================] - 0s 6ms/step

Top 10 recommendations similar to Pantai Cibobos (excluding the same name):
1. Karang Taraje Beach (Similarity: 2.594)
   Category: Tempat Wisata
   City: Lebak
   Rating: 4.6
   Price: 0
   Latitude: -6.990635213
   Longitude: 106.3262797

2. Tanjung Layar Beach (Similarity: 2.594)
   Category: Tempat Wisata
   City: Lebak
   Rating: 4.6
   Price: 0
   Latitude: -6.99373142
   Longitude: 106.3072142

3. Pantai Karang Songsong (Similarity: 2.591)
   Category: Tempat Wisata
   City: Lebak
   Rating: 4.6
   Price: 0
   Latitude: -6.887673946
   Longitude: 106.1192091

5. Negeri Di Atas Awan Citorek (Similarity: 2.590)
   Category: Tempat Wisata
   City: Lebak
   Rating: 4.6
   Price: 0
   Latitude: -6.741901544
   Longitude: 106.3319757

6. Wisata Kampung Marengo Baduy Luar (Similarity: 2.587)
   Category: Tempat Wisata
   City: Lebak
   Rating: 4.7
   Price: 0
   Latitude: -6.607700721
   Longitude: 106.222503

7. Tourism Village Bad